In [2]:
import robot
import map

import numpy as np
import gtsam
import matplotlib.pyplot as plt
import random

%matplotlib qt

## Move robot

In [2]:
car = robot.robot()

#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)

time = 0 #[s]
dt = 0.5 
timeFinal = 10

#keep history
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()

with plt.ion():
    #set graphics
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim(-2,2); ax.set_ylim(-1,3); 
    ax.set_xlabel('x'); ax.set_ylabel('y'); 
    ax.set_aspect('equal'); ax.grid()
    graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
    graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
    ax.legend(["ground truth","dead reckoning"])

    #run and plot simulation
    xcurrent_DR = hist_GT.copy()
    while time <= timeFinal:
        meas_dr,meas_dtheta = car.moveAndMeasureOdometrey((dr,dtheta))
        
        #dead reckoning integration
        xcurrent_DR[2] += meas_dtheta #update angle
        xcurrent_DR[0] += meas_dr * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_dr * np.sin(xcurrent_DR[2])
        
        #log history
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_GT = np.vstack([hist_GT,car.pose])

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(0.01)

## World Landmarks

In [9]:
np.random.seed(seed=5)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = {
                    'x':(np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                    'y':(np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                    'classLabel':random.choice(semantics),
                    }
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs@sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii]['cov'] = cov

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True)

In [4]:
cov

array([[ 0.08601365, -0.0048512 ],
       [-0.0048512 ,  0.01248669]])